# Домашнее задание №1

На семинаре мы проходили метрики, необходимо реализовать некоторые из них

In [19]:
import numpy as np
import pandas as pd
import math

In [7]:
data = pd.DataFrame({"user_id": ["u1","u2","u3"], 
              "recommended_list": [
                                      [143, 156, 1134, 991, 27],
                                      [1543, 3345, 533, 11, 43],
                                      [156, 3345, 10, 15, 1134]
                                  ],
             "bought_list": [
                             [156, 27],
                             [11, 43],
                             [1]
                            ],
                           
            "prices_recommended":[
                [456, 589, 135, 95, 569],
                [236, 102, 381, 165, 28],
                [981, 123, 653, 561, 78]
            ],
            "prices_bought":[
                [589, 569],
                [165, 43],
                [986]
            ]
        })
data

,user_id,recommended_list,bought_list,prices_recommended,prices_bought
0,u1,"[143, 156, 1134, 991, 27]","[156, 27]","[456, 589, 135, 95, 569]","[589, 569]"
1,u2,"[1543, 3345, 533, 11, 43]","[11, 43]","[236, 102, 381, 165, 28]","[165, 43]"
2,u3,"[156, 3345, 10, 15, 1134]",[1],"[981, 123, 653, 561, 78]",[986]


## Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


In [8]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    recommended_list = recommended_list[:k]
    flags = np.isin(recommended_list, bought_list)
    recall = flags.sum() / len(bought_list)

    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    prices_bought = np.array(prices_bought)
    
    flags = np.isin(recommended_list, bought_list)
    money_recall = (flags*prices_recommended).sum() / (bought_list*prices_bought).sum()
    
    return money_recall

In [9]:
data.apply(lambda x: recall_at_k(x[1], x[2]), 1)

0    1.0
1    1.0
2    0.0
dtype: float64

In [10]:
data.apply(lambda x: money_recall_at_k(x[1], x[2], x[3], x[4],k=5), 1)

0    0.010798
1    0.052675
2    0.000000
dtype: float64

## Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  MMR(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [15]:
def mrr_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    recommended_list = recommended_list[:k]
    rank = 1
    for i in range(len(recommended_list)):
        if np.isin(recommended_list[i],bought_list):
            rank = i+1
            break
        else:
            continue  
    rr_at_k = 1/rank

    return rr_at_k

In [16]:
data.apply(lambda x: mrr_at_k(x[1], x[2]), 1)

0    0.50
1    0.25
2    1.00
dtype: float64

## Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [20]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    recommended_list = recommended_list[:k]
    dcg_at_k = 0
    flags = np.isin(recommended_list,bought_list)
    for i in range(len(flags)):
        dcg_at_k += flags[i]/math.log2(i+2)
   
    idcg_at_k = 0
    for i in range(len(flags)):
        if flags[i]:
            idcg_at_k = 1/math.log2(i+2)
            break
        else:
            continue
        
    ndcg_at_k = dcg_at_k/idcg_at_k
    
    return ndcg_at_k

In [21]:
data.apply(lambda x: ndcg_at_k(x[1], x[2]), 1)

/tmp/ipykernel_419/754909008.py:19: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_at_k = dcg_at_k/idcg_at_k


0    1.613147
1    1.898244
2         NaN
dtype: float64